# 1. Cargar el set de datos en un dataframe de pandas.


In [3]:
# Espacio para realizar la importación de librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [4]:
ubicacionArchivo = "diabetes.csv"
# Carga de un CSV a un archivo tipo DataFrame por medio de pandas.
df = pd.read_csv(ubicacionArchivo)

# 2. Mostrar los mejores resultados para ambos problemas anteriores (regresión y clasificación).

In [5]:
from sklearn.model_selection import train_test_split

# Definimos las variables X e y
X = df.drop(['Outcome', 'Age'], axis=1)  
y = df['Age']  

# Dividimos el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2033)

# Imprimimos las formas de los conjuntos de entrenamiento y prueba
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

Forma de X_train: (614, 7)
Forma de X_test: (154, 7)
Forma de y_train: (614,)
Forma de y_test: (154,)


In [6]:
#Para la etiquta "Age"

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


# Definimos los parámetros 
param_grid = {
    'max_depth': [3, 10, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3]
}

# Instanciamos el modelo de árbol de decisión
regression_tree = DecisionTreeRegressor(random_state=2033)

# Realizamos la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(regression_tree, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_

# Entrenamos el modelo con los mejores parámetros
best_regressor = DecisionTreeRegressor(**best_params, random_state=2033)
best_regressor.fit(X_train, y_train)

# Hacemos predicciones con el conjunto de pruebas
y_pred = best_regressor.predict(X_test)

# Calculamos el error cuadrático medio (MSE) con los mejores parámetros
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

# Imprimimos los mejores parámetros encontrados
print("Mejores hiperparámetros:", best_params)

Error cuadrático medio (MSE): 90.695176249044
Mejores hiperparámetros: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [7]:
# Para la etiqueta "Outcome"

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint

# Definimos los hiperparámetros y sus rangos
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(1, 30),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20)
}

# Instanciamos el árbol de decisión para clasificación
classifier = DecisionTreeClassifier(random_state=2033)

# Realizamos la búsqueda aleatoria para encontrar los mejores hiperparámetros
random_search = RandomizedSearchCV(classifier, param_distributions=param_dist, n_iter=100, cv=5, random_state=2033)
random_search.fit(X_train, y_train)

# Obtenemos los mejores hiperparámetros
best_params = random_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Obtenemos el mejor modelo
best_model = random_search.best_estimator_

# Hacemos predicciones con el conjunto de pruebas
y_pred = best_model.predict(X_test)

# Calculamos la precisión (accuracy) del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión (Accuracy):", accuracy)

C:\Users\sams\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 11, 'min_samples_split': 7}
Precisión (Accuracy): 0.08441558441558442


# 3. Importe las librerías para el VotingClassifier y el RandomForest (tanto para regresión como para clasificación).

In [8]:
# Para el VotingClassifier
from sklearn.ensemble import VotingClassifier

# Para el RandomForest de regresión
from sklearn.ensemble import RandomForestRegressor

# Para el RandomForest de clasificación
from sklearn.ensemble import RandomForestClassifier

# 4. Entrene un modelo de votación para regresión con 3 modelos de regresión de su preferencia (por ejemplo: regresión lineal, árbol de decisión, KNeighborRegressor), utilice las mismas muestras de “X” y “y” que en la actividad anterior.

In [26]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Definir los estimadores
estimators = [('linear', LinearRegression()), ('tree', DecisionTreeRegressor()), ('knn', KNeighborsRegressor())]

# Instanciar el modelo de votación para regresión
voting_regressor = VotingRegressor(estimators)

# Entrenar el modelo de votación
voting_regressor.fit(X_train, y_train)



VotingRegressor(estimators=[('linear', LinearRegression()),
                            ('tree', DecisionTreeRegressor()),
                            ('knn', KNeighborsRegressor())])

# 5. Pruebe el modelo de votación con la muestra de pruebas y evalúe con MSE.

In [29]:
# Hacer predicciones con el conjunto de pruebas
y_pred_voting = voting_regressor.predict(X_test)

# Calcular el error cuadrático medio (MSE) del modelo de votación
mse_voting = mean_squared_error(y_test, y_pred_voting)
print("Error cuadrático medio (MSE) del modelo de votación:", mse_voting)

Error cuadrático medio (MSE) del modelo de votación: 88.14423538206117


# 6. ¿Tuvo mejores o peores resultados?, ¿qué tanto tardó en entrenarse?



# 7. Entrene un modelo de votación para clasificación con 3 modelos de clasificación de su preferencia (por ejemplo: regresión logística, árbol de decisión y KNeighborClassifier), utilice las mismas muestras de “X” y “y” que la actividad anterior.


In [31]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Supongamos que X, y representan tus datos y objetivos de clasificación respectivamente

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2033)

# Definir los estimadores
estimators = [('logistic', LogisticRegression()), ('tree', DecisionTreeClassifier()), ('knn', KNeighborsClassifier())]

# Instanciar el modelo de votación para clasificación
voting_classifier = VotingClassifier(estimators)

# Entrenar el modelo de votación
voting_classifier.fit(X_train, y_train)

# Hacer predicciones con el conjunto de pruebas
y_pred = voting_classifier.predict(X_test)

# Calcular la precisión (accuracy) del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión (Accuracy) del modelo de votación:", accuracy)


Precisión (Accuracy) del modelo de votación: 0.08441558441558442


C:\Users\sams\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# 8. Pruebe el modelo de votación con la muestra de pruebas y evalúe con Accuracy_Score y F1_Score.

In [34]:
from sklearn.metrics import accuracy_score, f1_score

# Supongamos que X_test y y_test son tus conjuntos de prueba

# Hacer predicciones con el conjunto de pruebas
y_pred_voting = voting_classifier.predict(X_test)

# Calcular la precisión (accuracy) del modelo
accuracy_voting = accuracy_score(y_test, y_pred_voting)
print("Precisión (Accuracy) del modelo de votación:", accuracy_voting)

# Calcular la puntuación F1 del modelo con average='micro'
f1_voting_micro = f1_score(y_test, y_pred_voting, average='micro')
print("Puntuación F1 (micro) del modelo de votación:", f1_voting_micro)

# Calcular la puntuación F1 del modelo con average='macro'
f1_voting_macro = f1_score(y_test, y_pred_voting, average='macro')
print("Puntuación F1 (macro) del modelo de votación:", f1_voting_macro)

# Calcular la puntuación F1 del modelo con average='weighted'
f1_voting_weighted = f1_score(y_test, y_pred_voting, average='weighted')
print("Puntuación F1 (weighted) del modelo de votación:", f1_voting_weighted)


Precisión (Accuracy) del modelo de votación: 0.08441558441558442
Puntuación F1 (micro) del modelo de votación: 0.08441558441558442
Puntuación F1 (macro) del modelo de votación: 0.01229129550506882
Puntuación F1 (weighted) del modelo de votación: 0.041393060720791824


# 9. ¿Tuvo mejores o peores resultados?, ¿qué tanto tardó en entrenarse?

# 10. Entrene un modelo de bosque aleatorio para clasificación, puede emplear los hiperparámetros que considere adecuados (de esto dependerá la eficiencia del modelo), para el entrenamiento se debe usar la misma muestra que en los casos anteriores.


In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2033)

# Definir y entrenar el modelo de Bosque Aleatorio para clasificación
random_forest_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=2033)
random_forest_classifier.fit(X_train, y_train)

# Hacer predicciones con el conjunto de pruebas
y_pred = random_forest_classifier.predict(X_test)

# Calcular la precisión (accuracy) del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión (Accuracy) del modelo de Bosque Aleatorio:", accuracy)


Precisión (Accuracy) del modelo de Bosque Aleatorio: 0.07792207792207792


# 11. Se debe evaluar el modelo con F1_Score y Accuracy_Score.

In [37]:

# Hacer predicciones con el conjunto de pruebas
y_pred_rf = random_forest_classifier.predict(X_test)

# Calcular la precisión (accuracy) del modelo
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Precisión (Accuracy) del modelo de Bosque Aleatorio:", accuracy_rf)

# Calcular la puntuación F1 del modelo con average='micro'
f1_rf_micro = f1_score(y_test, y_pred_rf, average='micro')
print("Puntuación F1 (micro) del modelo de Bosque Aleatorio:", f1_rf_micro)

# Calcular la puntuación F1 del modelo con average='macro'
f1_rf_macro = f1_score(y_test, y_pred_rf, average='macro')
print("Puntuación F1 (macro) del modelo de Bosque Aleatorio:", f1_rf_macro)

# Calcular la puntuación F1 del modelo con average='weighted'
f1_rf_weighted = f1_score(y_test, y_pred_rf, average='weighted')
print("Puntuación F1 (weighted) del modelo de Bosque Aleatorio:", f1_rf_weighted)

Precisión (Accuracy) del modelo de Bosque Aleatorio: 0.07792207792207792
Puntuación F1 (micro) del modelo de Bosque Aleatorio: 0.07792207792207792
Puntuación F1 (macro) del modelo de Bosque Aleatorio: 0.03254388677321008
Puntuación F1 (weighted) del modelo de Bosque Aleatorio: 0.07422544702961655


# 12. ¿Qué tal se comportó?, ¿mejoró o empeoró con respecto a los modelos anteriores?,  ¿tomó más o menos tiempo para entrenarse?


# 13. Entrene nuevamente el bosque aleatorio, pero cambiando los hiperparámetros de manera radical. ¿Qué tal se comportó el modelo?


In [38]:
from sklearn.model_selection import train_test_split

# Supongamos que X, y representan tus datos y objetivos de clasificación respectivamente

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2033)

# Definir y entrenar el modelo de Bosque Aleatorio con hiperparámetros diferentes
random_forest_classifier_new = RandomForestClassifier(n_estimators=50, max_depth=5, min_samples_split=5, random_state=2033)
random_forest_classifier_new.fit(X_train, y_train)

# Hacer predicciones con el conjunto de pruebas
y_pred_new = random_forest_classifier_new.predict(X_test)

# Calcular la precisión (accuracy) del modelo
accuracy_new = accuracy_score(y_test, y_pred_new)
print("Precisión (Accuracy) del modelo de Bosque Aleatorio (nuevos hiperparámetros):", accuracy_new)

# Calcular la puntuación F1 del modelo con average='weighted'
f1_new = f1_score(y_test, y_pred_new, average='weighted')
print("Puntuación F1 (weighted) del modelo de Bosque Aleatorio (nuevos hiperparámetros):", f1_new)


Precisión (Accuracy) del modelo de Bosque Aleatorio (nuevos hiperparámetros): 0.09090909090909091
Puntuación F1 (weighted) del modelo de Bosque Aleatorio (nuevos hiperparámetros): 0.057527143088640424


# 14. Escriba sus conclusiones obtenidas al utilizar modelos de ensamble.